# Benchmarking Kronecker.jl

In this notebook, we illustrate the basic functionality of `Kronecker.jl` and compare the runtime with native Julia code.

In [1]:
using Kronecker, BenchmarkTools, LinearAlgebra

┌ Info: Recompiling stale cache file /Users/michielstock/.julia/compiled/v1.1/Kronecker/6eght.ji for Kronecker [2c470bb0-bcc8-11e8-3dad-c9649493f05e]
└ @ Base loading.jl:1184


## Basic functionality

In [2]:
# define some matrices

A = randn(4, 4);
B = Array{Float64, 2}([1 2 3;
     4 5 6;
     7 -2 9]);
C = rand(5, 6);

v = rand(12);

A Kronecker product between two matrices (of the same type) is constructed using $\otimes$, typed as `\otimes TAB`. It is **not** directly evaluated unless we use `collect`.

In [3]:
K = A ⊗ B

Kronecker system of the form A ⊗ B

In [4]:
collect(K)

12×12 Array{Float64,2}:
  0.435692   0.871385    1.30708  …  0.156728    0.313455   0.470183
  1.74277    2.17846     2.61415     0.626911    0.783638   0.940366
  3.04985   -0.871385    3.92123     1.09709    -0.313455   1.41055 
  1.01061    2.02122     3.03183     0.995015    1.99003    2.98504 
  4.04244    5.05305     6.06366     3.98006     4.97507    5.97009 
  7.07427   -2.02122     9.09549  …  6.9651     -1.99003    8.95513 
  1.47839    2.95678     4.43516     0.0382776   0.0765552  0.114833
  5.91355    7.39194     8.87033     0.15311     0.191388   0.229666
 10.3487    -2.95678    13.3055      0.267943   -0.0765552  0.344499
 -1.33299   -2.66597    -3.99896     0.396531    0.793061   1.18959 
 -5.33194   -6.66493    -7.99791  …  1.58612     1.98265    2.37918 
 -9.3309     2.66597   -11.9969      2.77571    -0.793061   3.56878 

In [5]:
K[3, 4]  # indexing works, computes result directly

-0.049554120708304394

Compare with Julia native result.

In [6]:
X = kron(A, B)

12×12 Array{Float64,2}:
  0.435692   0.871385    1.30708  …  0.156728    0.313455   0.470183
  1.74277    2.17846     2.61415     0.626911    0.783638   0.940366
  3.04985   -0.871385    3.92123     1.09709    -0.313455   1.41055 
  1.01061    2.02122     3.03183     0.995015    1.99003    2.98504 
  4.04244    5.05305     6.06366     3.98006     4.97507    5.97009 
  7.07427   -2.02122     9.09549  …  6.9651     -1.99003    8.95513 
  1.47839    2.95678     4.43516     0.0382776   0.0765552  0.114833
  5.91355    7.39194     8.87033     0.15311     0.191388   0.229666
 10.3487    -2.95678    13.3055      0.267943   -0.0765552  0.344499
 -1.33299   -2.66597    -3.99896     0.396531    0.793061   1.18959 
 -5.33194   -6.66493    -7.99791  …  1.58612     1.98265    2.37918 
 -9.3309     2.66597   -11.9969      2.77571    -0.793061   3.56878 

In [7]:
@btime kron($A, $B)  # computational intensive!

  215.555 ns (1 allocation: 1.22 KiB)


12×12 Array{Float64,2}:
  0.435692   0.871385    1.30708  …  0.156728    0.313455   0.470183
  1.74277    2.17846     2.61415     0.626911    0.783638   0.940366
  3.04985   -0.871385    3.92123     1.09709    -0.313455   1.41055 
  1.01061    2.02122     3.03183     0.995015    1.99003    2.98504 
  4.04244    5.05305     6.06366     3.98006     4.97507    5.97009 
  7.07427   -2.02122     9.09549  …  6.9651     -1.99003    8.95513 
  1.47839    2.95678     4.43516     0.0382776   0.0765552  0.114833
  5.91355    7.39194     8.87033     0.15311     0.191388   0.229666
 10.3487    -2.95678    13.3055      0.267943   -0.0765552  0.344499
 -1.33299   -2.66597    -3.99896     0.396531    0.793061   1.18959 
 -5.33194   -6.66493    -7.99791  …  1.58612     1.98265    2.37918 
 -9.3309     2.66597   -11.9969      2.77571    -0.793061   3.56878 

Basic algebraic properties can be computed, but much faster. Note that we compare using the **precomputed** Kronecker product, something which is not even possible for realistic matrices.

### Trace

In [8]:
@btime tr($A ⊗ $B)

  14.897 ns (1 allocation: 32 bytes)


4.859198012490642

In [9]:
@btime tr($X)  # this excludes computing the Kronecker product

  10.412 ns (0 allocations: 0 bytes)


4.859198012490642

### Determinant

In [10]:
@btime det($A ⊗ $B)

  595.056 ns (7 allocations: 688 bytes)


1.0059371916772203e10

In [11]:
@btime det($X)

  1.199 μs (3 allocations: 1.42 KiB)


1.0059371916772207e10

### Matrix inverse

In [12]:
@btime inv($A ⊗ $B)

  1.341 μs (11 allocations: 4.52 KiB)


Kronecker system of the form A ⊗ B

In [13]:
@btime collect(inv($A ⊗ $B))  # inv returns an implict Kronecker structure

  1.577 μs (11 allocations: 5.70 KiB)


12×12 Array{Float64,2}:
 -0.0829946    0.0349451   0.00436814  …  -0.108404    -0.0135505 
 -0.00873627   0.0174725  -0.00873627     -0.0542018    0.0271009 
  0.06261     -0.0232967   0.00436814      0.0722691   -0.0135505 
  0.463642    -0.195218   -0.0244022       0.0747643    0.00934554
  0.0488045   -0.0976089   0.0488045       0.0373821   -0.0186911 
 -0.349765     0.130145   -0.0244022   …  -0.0498429    0.00934554
 -0.738635     0.311004    0.0388755       0.00863079   0.00107885
 -0.077751     0.155502   -0.077751        0.0043154   -0.0021577 
  0.557216    -0.207336    0.0388755      -0.00575386   0.00107885
  0.454709    -0.191456   -0.023932        0.231532     0.0289416 
  0.0478641   -0.0957281   0.0478641   …   0.115766    -0.0578831 
 -0.343026     0.127638   -0.023932       -0.154355     0.0289416 

In [14]:
@btime inv($X)

  2.530 μs (5 allocations: 7.58 KiB)


12×12 Array{Float64,2}:
 -0.0829946    0.0349451   0.00436814  …  -0.108404    -0.0135505 
 -0.00873627   0.0174725  -0.00873627     -0.0542018    0.0271009 
  0.06261     -0.0232967   0.00436814      0.0722691   -0.0135505 
  0.463642    -0.195218   -0.0244022       0.0747643    0.00934554
  0.0488045   -0.0976089   0.0488045       0.0373821   -0.0186911 
 -0.349765     0.130145   -0.0244022   …  -0.0498429    0.00934554
 -0.738635     0.311004    0.0388755       0.00863079   0.00107885
 -0.077751     0.155502   -0.077751        0.0043154   -0.0021577 
  0.557216    -0.207336    0.0388755      -0.00575386   0.00107885
  0.454709    -0.191456   -0.023932        0.231532     0.0289416 
  0.0478641   -0.0957281   0.0478641   …   0.115766    -0.0578831 
 -0.343026     0.127638   -0.023932       -0.154355     0.0289416 

### Matrix vector multiplication

One of the most important features is the efficient multiplication of a Kronecker product with a vector.

In [15]:
@btime ($A ⊗ $B) * $v

  505.902 ns (9 allocations: 784 bytes)


12-element Array{Float64,1}:
  3.8204226925149998 
 10.879185005408887  
 10.766707076363273  
 -0.5747672593272235 
 -0.20731224414692256
  7.682539892084371  
 12.535484172885637  
 31.303135249155698  
 25.14952036237209   
  7.300407758930517  
 17.795455719989135  
  0.8126811170795302 

In [16]:
@btime kron($A, $B) * $v

  330.686 ns (2 allocations: 1.39 KiB)


12-element Array{Float64,1}:
  3.8204226925149998 
 10.879185005408885  
 10.766707076363275  
 -0.5747672593272233 
 -0.20731224414692395
  7.68253989208437   
 12.535484172885639  
 31.303135249155698  
 25.149520362372094  
  7.3004077589305165 
 17.795455719989132  
  0.8126811170795301 

Using a somewhat larger example...

In [17]:
C = rand(50, 60);
D = randn(80, 50);
u = collect(1.0:(50*60));

In [18]:
@btime (C ⊗ D) * u;

  25.757 μs (12 allocations: 82.55 KiB)


In [19]:
@btime kron(C, D) * u;

  23.897 ms (4 allocations: 91.58 MiB)


In [20]:
all(((C ⊗ D) * u) ≈ (kron(C, D) * u))

true

## Indexed Krocker systems

In many applications, one has to deal with incomplete Kronecker systems. Indexed systems are supported and Kronecker system-vector multiplications are supported using the generalized vec trick.

In [21]:
v = rand(10);

# sizes
a, b = 4, 8;
c, d = 5, 9;

# matrices
M = randn(a, b);
N = rand(c, d);

# indices
p = rand(1:a, 6);
q = rand(1:c, 6);

r = rand(1:b, 10);
t = rand(1:d, 10);

In [22]:
kronprod = N ⊗ M
ikp = kronprod[p,q,r,t]  # subpart of the Kronecker system

Indexed Kronecker system of the form A ⊗ B


In [23]:
collect(ikp)

6×10 Array{Float64,2}:
 -0.79884    0.304819   0.158234   …  -1.33391    1.21746   -0.00690708 
 -0.850978  -0.613535  -0.966669      -0.305549   0.307198   0.023155   
 -0.850978  -0.613535  -0.966669      -0.305549   0.307198   0.023155   
 -0.450945   0.352031   0.0414813     -0.349686   1.40602   -0.00536021 
 -0.794386   0.105433   0.0775616     -0.653843   0.421104  -0.000757542
  0.651077  -0.551469  -0.229731   …  -0.248129  -1.18444    0.688593   

In [24]:
# computing this subsystem explicitely
subsystem = kron(N, M)[a * (q .- 1) .+ p, b * (t .- 1) .+ r]

6×10 Array{Float64,2}:
 -0.79884    0.304819   0.158234   …  -1.33391    1.21746   -0.00690708 
 -0.850978  -0.613535  -0.966669      -0.305549   0.307198   0.023155   
 -0.850978  -0.613535  -0.966669      -0.305549   0.307198   0.023155   
 -0.450945   0.352031   0.0414813     -0.349686   1.40602   -0.00536021 
 -0.794386   0.105433   0.0775616     -0.653843   0.421104  -0.000757542
  0.651077  -0.551469  -0.229731   …  -0.248129  -1.18444    0.688593   

### Multiplication

In [25]:
@btime $ikp * $v

  545.663 ns (4 allocations: 560 bytes)


6-element Array{Float64,1}:
  1.5470074607820292 
 -0.19497052505120305
 -0.19497052505120305
  1.536201311420557  
  0.8908172865470616 
 -1.2290139916080636 

In [26]:
@btime kron($N, $M)[$a * ($q .- 1) .+ $p, $b * ($t .- 1) .+ $r] * $v

  2.225 μs (10 allocations: 12.94 KiB)


6-element Array{Float64,1}:
  1.5470074607820292 
 -0.19497052505120305
 -0.19497052505120305
  1.536201311420557  
  0.8908172865470617 
 -1.2290139916080638 

In [27]:
@btime $subsystem * $v  # to be fair, if the system fits into memory, it's faster

  95.658 ns (1 allocation: 128 bytes)


6-element Array{Float64,1}:
  1.5470074607820292 
 -0.19497052505120305
 -0.19497052505120305
  1.536201311420557  
  0.8908172865470617 
 -1.2290139916080638 

## Shifted Kronecker systems

In [28]:
# again, define some matrices
# here, we explicitly work with symmetric matrices 

A = Symmetric(rand(10, 10));
B = Symmetric(rand(30, 30));
v = rand(300);

### Solving a shifted Kronecker system

In [29]:
@btime ($A ⊗ $B + 2I) \ $v

  203.134 μs (61 allocations: 66.17 KiB)


300-element Array{Float64,1}:
 -3.274721338014931  
  1.1298259144726328 
  4.408310687892794  
 -3.463691981306303  
  3.6794707750568527 
 -5.569585446321778  
  7.14918252998517   
  0.33390940919512224
  3.2394917001859453 
  1.1118478984385893 
  1.5210761959924386 
 -1.3598788195228384 
  0.4389997075925702 
  ⋮                  
 -5.088008751434819  
 -2.2531278065905416 
  2.171994142861282  
  4.559857625981706  
 -2.2662613500007267 
  1.9581125757052313 
 -4.449356074264652  
  2.8965707578516615 
 -1.2330881051338751 
  3.044323471859763  
 -1.124142081665091  
 -2.046043173408333  

In [30]:
@btime (kron($A, $B) + 2I) \ $v

  1.238 ms (9 allocations: 2.07 MiB)


300-element Array{Float64,1}:
 -3.2747213380134044 
  1.1298259144717184 
  4.4083106878923335 
 -3.4636919813035876 
  3.679470775055062  
 -5.569585446318218  
  7.149182529982481  
  0.33390940919652984
  3.239491700182582  
  1.1118478984408708 
  1.5210761959914265 
 -1.3598788195238478 
  0.438999707591998  
  ⋮                  
 -5.088008751430712  
 -2.2531278065898266 
  2.1719941428607537 
  4.559857625979678  
 -2.2662613500002022 
  1.9581125757043876 
 -4.449356074262929  
  2.8965707578493536 
 -1.2330881051319902 
  3.044323471857813  
 -1.1241420816634822 
 -2.0460431734057254 